In [1]:
import sys
sys.path.append("..")

# Load the model from checkpoint

After fine-tuning from [train.py](./train.py), checkpoint would be saved in [checkpoints/last.ckpt](./checkpoints/)

In [3]:
from transformers import (
    Wav2Vec2ForPreTraining,
    Wav2Vec2CTCTokenizer,
    Wav2Vec2FeatureExtractor,
)
from finetuning.wav2vec2 import SpeechRecognizer


def load_model(ckpt_path: str):
    model_name = "nguyenvulebinh/wav2vec2-base-vietnamese-250h"

    wav2vec2 = Wav2Vec2ForPreTraining.from_pretrained(model_name)
    tokenizer = Wav2Vec2CTCTokenizer.from_pretrained(model_name)
    feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(model_name)

    model = SpeechRecognizer.load_from_checkpoint(
        ckpt_path,
        wav2vec2=wav2vec2,
        tokenizer=tokenizer,
        feature_extractor=feature_extractor,
    )

    return model


recognizer = load_model("./checkpoints/last.ckpt")


/home/hoang/.local/lib/python3.10/site-packages/transformers/configuration_utils.py:369: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Some weights of the model checkpoint at nguyenvulebinh/wav2vec2-base-vietnamese-250h were not used when initializing Wav2Vec2ForPreTraining: ['lm_head.bias', 'lm_head.weight']
- This IS expected if you are initializing Wav2Vec2ForPreTraining from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForPreTraining from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model

# Validation

## VLSP2020 Dataset

In [9]:
from src.datamodule import VLSP2020TarDataset
from src.datamodule.vlsp2020 import get_dataloader

dts = VLSP2020TarDataset("data/vlsp2020_val_set.tar").load()
val_loader = get_dataloader(dts, batch_size=2, return_transcript=True, num_workers=2)

In [10]:
from pytorch_lightning import Trainer
import torch

torch.cuda.empty_cache()
metrics = Trainer(accelerator="gpu", devices=1).validate(model=recognizer, dataloaders=val_loader)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Validation: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     Validate metric           DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
         val/cer            0.07352624833583832
         val/wer            0.15021772682666779
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


## VIVOS Dataset

In [15]:
from datasets import load_dataset

dataset = load_dataset("vivos", split="test")

Generating train split:   0%|          | 0/11660 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/760 [00:00<?, ? examples/s]

Dataset vivos downloaded and prepared to /root/.cache/huggingface/datasets/vivos/default/1.1.0/4872f55990894df809a7d41dc64c483303a4a6f495c50b6e434632190c1b5eeb. Subsequent calls will reuse this data.


In [25]:
dataset = dataset.remove_columns(["speaker_id", "path"]).with_format("torch")
dataset

Dataset({
    features: ['audio', 'sentence'],
    num_rows: 760
})

In [32]:
from torch.utils.data import DataLoader

def collate_fn(batch):
    waveforms, transcripts = [], []

    for item in batch:
        waveforms.append(item['audio']['array'])
        transcripts.append(item['sentence'].lower())

    return transcripts, waveforms

val_loader = DataLoader(
    dataset,
    batch_size=2,
    num_workers=2,
    collate_fn=collate_fn
)

In [34]:
torch.cuda.empty_cache()
metrics = Trainer(accelerator="gpu", devices=1).validate(model=recognizer, dataloaders=val_loader)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Validation: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     Validate metric           DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
         val/cer            0.04655877500772476
         val/wer            0.1258229911327362
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


# Inference

In [5]:
import requests
import torchaudio
from IPython.display import Audio, display


def get_audio(path: str):
    default_sample_rate = 16000
    if path.startswith("http"):
        with requests.get(path, stream=True) as r:
            waveform, sample_rate = torchaudio.load(r.raw)
    else:
        waveform, sample_rate = torchaudio.load(path)

    waveform = torchaudio.transforms.Resample(sample_rate, default_sample_rate)(
        waveform
    )

    return waveform


def play_audio(waveform: torch.Tensor, sample_rate: int = 16000):
    return display(Audio(waveform.cpu().numpy(), rate=sample_rate))


In [7]:
waveform = get_audio("https://github.com/NTT123/Vietnamese-Text-To-Speech-Dataset/blob/master/000000.wav?raw=true")
print(waveform.shape)

play_audio(waveform)
print(recognizer.predict(waveform))

torch.Size([1, 121859])


['hai đây tức là một khè ăn mày vậy anh ta chưa kịp quay đi thì đã thấy mấy con chó vàng chảy sồng sộc ra cứ nhảy sổ vào chân anh']
